# Ozone - MSR

## Import Packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot, utils

plt.style.use("seaborn-v0_8-notebook")

## Define parameters

In [ ]:
# Regions
region_slices = {
    "global": {"lat_slice": slice(-90, 90), "lon_slice": slice(0, 360)},
    "tropics": {"lat_slice": slice(-25, 25), "lon_slice": slice(0, 360)},
    "NH mid-latitudes": {"lat_slice": slice(30, 60), "lon_slice": slice(0, 360)},
    "SH mid-latitudes": {"lat_slice": slice(-30, -60), "lon_slice": slice(0, 360)},
    "NH polar": {"lat_slice": slice(60, 90), "lon_slice": slice(0, 360)},
    "SH polar": {"lat_slice": slice(-60, -90), "lon_slice": slice(0, 360)},
}

# Periods
version_timeseries = {
    "v0020": {"start": "1979-01", "stop": "2018-12"},
    "v0023": {"start": "1979-01", "stop": "2018-12"},
    "v0024": {"start": "2018-12", "stop": "2021-12"},
    "v0025": {"start": "2022-01", "stop": "2022-12"},
}
version_maps = {
    "v0023": {"start": "1979-01", "stop": "2018-12"},
    "v0024": {"start": "2019-01", "stop": "2021-12"},
}

## Define request

In [ ]:
collection_id = "satellite-ozone-v1"

common_request = {
    "format": "zip",
    "processing_level": "level_4",
    "vertical_aggregation": "total_column",
    "sensor": "msr",
    "variable": "atmosphere_mole_content_of_ozone",
}

requests_timeseries = {
    version: download.update_request_date(
        common_request | {"version": version},
        stringify_dates=True,
        **update_kwargs,
    )
    for version, update_kwargs in version_timeseries.items()
}

requests_maps = []
for version, update_kwargs in version_maps.items():
    requests_maps.extend(
        download.update_request_date(
            common_request | {"version": version},
            stringify_dates=True,
            **update_kwargs,
        )
    )

## Cached functions

In [ ]:
def fix_coords(ds):
    ds = ds.swap_dims(
        {
            short: long
            for short, long in zip(["Lat", "Lon"], ["latitude", "longitude"])
            if short in ds.dims
        }
    )
    time = ds.attrs["Ozone_field_date"]
    if isinstance(time, np.ndarray):
        time = "-".join(ds.attrs["Ozone_field_date"].astype(str))
    ds = ds.assign_coords(time=[pd.to_datetime(time)])
    return ds


def spatial_weighted_mean(ds, lon_slice, lat_slice):
    ds = utils.regionalise(ds, lon_slice=lon_slice, lat_slice=lat_slice)
    return diagnostics.spatial_weighted_mean(ds)

## Download and transform timeseries

In [ ]:
kwargs = {
    "collection_id": collection_id,
    "chunks": {"year": 1},
    "decode_times": False,
    "preprocess": fix_coords,
}

datasets = []
for version, requests in requests_timeseries.items():
    for region, transform_func_kwargs in region_slices.items():
        print(f"{version=} {region=}")
        ds = download.download_and_transform(
            requests=requests,
            transform_func=spatial_weighted_mean,
            transform_func_kwargs=transform_func_kwargs,
            **kwargs,
        )
        datasets.append(ds.expand_dims(version=[version], region=[region]))
ds_timeseries = xr.merge(datasets)

## Download and transform maps

In [ ]:
ds_annual = download.download_and_transform(
    requests=requests_maps,
    transform_func=diagnostics.annual_weighted_mean,
    transform_chunks=False,
    **kwargs,
)
ds_monthly = download.download_and_transform(
    requests=requests_maps,
    transform_func=diagnostics.monthly_weighted_mean,
    transform_chunks=False,
    **kwargs,
)

## Plot timeseries

In [ ]:
facet = ds_timeseries["total_ozone_column"].plot(
    col="region", hue="version", col_wrap=2
)
for ax in facet.axs.flatten():
    ax.grid()
facet.fig.autofmt_xdate(rotation=45)

## Plot monthly

In [ ]:
facet = plot.projected_map(
    ds_monthly["total_ozone_column"],
    col="month",
    col_wrap=3,
    projection=ccrs.Robinson(),
)

## Plot annual

In [ ]:
facet = plot.projected_map(
    ds_monthly["total_ozone_column"],
    col="year",
    col_wrap=6,
    projection=ccrs.SouthPolarStereo(central_longitude=0),
)